In [7]:
%cd ..

e:\projects\Chest X-Ray


In [9]:
import pandas as pd
data_map = pd.read_csv(r'data\balanced_data.csv')
data_map.head()

,imgs,clas,split,resized
0,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0001-0001.jpeg
1,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0003-0001.jpeg
2,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0005-0001.jpeg
3,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0006-0001.jpeg
4,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0007-0001.jpeg


In [10]:
from sklearn.preprocessing import LabelEncoder
data_map["clas_enc"]= LabelEncoder().fit_transform(data_map["clas"])

In [11]:
data_map

,imgs,clas,split,resized,clas_enc
0,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0001-0001.jpeg,0
1,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0003-0001.jpeg,0
2,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0005-0001.jpeg,0
3,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0006-0001.jpeg,0
4,load_data\chest_xray\chest_xray\test\NORMAL\IM...,NORMAL,test,load_data/resized/s\test_NORMAL_IM-0007-0001.jpeg,0
...,...,...,...,...,...
5851,load_data\chest_xray\chest_xray\val\PNEUMONIA\...,PNEUMONIA,val,load_data/resized/s\val_PNEUMONIA_person1949_b...,1
5852,load_data\chest_xray\chest_xray\val\PNEUMONIA\...,PNEUMONIA,val,load_data/resized/s\val_PNEUMONIA_person1950_b...,1
5853,load_data\chest_xray\chest_xray\val\PNEUMONIA\...,PNEUMONIA,val,load_data/resized/s\val_PNEUMONIA_person1951_b...,1
5854,load_data\chest_xray\chest_xray\val\PNEUMONIA\...,PNEUMONIA,val,load_data/resized/s\val_PNEUMONIA_person1952_b...,1


# torch

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image

In [12]:
from torchvision import transforms, models

In [13]:


class XRayDataset(Dataset):
    def __init__(self, df_split, transform=None):
        """
        Args:
            df (pd.DataFrame): DataFrame يحتوي على مسارات الصور والفئات.
            split (str): نوع التقسيم ('train', 'val', 'test').
            transform (callable, optional): تحويلات سيتم تطبيقها على الصور. Defaults to None.
        """
        self.df_split = df_split
        self.transform = transform

    def __len__(self):
        """
        Returns:
            int: عدد الصور في مجموعة البيانات الفرعية المحددة.
        """
        return len(self.df_split)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): فهرس الصورة المطلوبة.

        Returns:
            tuple: (image, label) حيث image هو Tensor يمثل الصورة و label هو Tensor يمثل الفئة.
        """
        img_path = self.df_split.iloc[idx]['resized']
        clas_name = self.df_split.iloc[idx]['clas_enc']
        image = Image.open(img_path).convert('RGB')  # تحويل الصورة إلى تدرجات الرمادي
        image = self.transform(image)

        return image, clas_name

In [14]:
train_dataset = XRayDataset(data_map[data_map['split'] == 'train'],transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
]))
val_dataset = XRayDataset(data_map[data_map['split'] == 'val'],transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
]))

In [27]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
# تحميل نموذج ResNet18 مسبقًا
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)


In [16]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Train

In [17]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
device

device(type='cpu')

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss=nn.CrossEntropyLoss()

In [29]:
from tqdm import tqdm

In [24]:
train_dataset
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)


In [26]:
train_dataset

In [30]:
for epoch in range(5):
    model.train()
    running_loss = 0
    for images, labels in tqdm(train_loader,total=len(train_loader)):
        # نقل البيانات إلى الجهاز المناسب (CPU أو GPU)
        images = images.to(device)
        labels = labels.to(device)
        
          # تحديث الأوزان
        optimizer.zero_grad()
        predictions = model(images)
        loss_value = loss(predictions, labels)
        loss_value.backward()
        optimizer.step()
        running_loss += loss_value.item()
    print(f'Epoch {epoch+1}/{5}, Loss: {running_loss/len(train_loader):.4f}')
    

  0%|          | 0/163 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'load_data/resized/s\\train_PNEUMONIA_person581_bacteria_2400.jpeg'

In [46]:
#save the model
torch.save(model.state_dict(), 'model.pth')